# M2 Rigid Body Position small range
 
This Jupyter notebook performs the Rigid Body Position (RBP) test of the M2 in the context of the M2 minimum functionality check.
Each M2 RBP Degree of Freedom (DoF) is actuated individually with RBP command of small amplitude and the evolution of the measured forces is observed. A conditional loop ensures that the maximum forces and related errors remain below the fault threshold.

## Import Modules

In [ ]:
import asyncio
import yaml
import numpy as np
import pandas as pd
import sys
import select
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

from lsst.ts.aos.utils import DiagnosticsM2, EfdName

## Prepare the M2 CSC and put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
"""    
DISABLED = 1
ENABLED = 2
FAULT = 3
OFFLINE = 4
STANDBY = 5

Standby  -->  Disable  -->  Enabled

await m2.cmd_start.set_start(timeout=30)
await m2.cmd_enable.set_start(timeout=200)

3) To Stop, do this:    Enabled  -->  Disable  -->  Standby

await m2.cmd_disable.set_start(timeout=30)
await m2.cmd_standby.set_start(timeout=30) """ 

In [ ]:
# get status
state = m2.evt_summaryState.get()
print(state)

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=30)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=450)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=460)

## Command RBP displacement

In [ ]:
import time

async def move_m2_rbp(axis: str):
    start = input('Start?')
    if start != 'y':
        return
    
    position = input('Position?')
    if len(position) == 0:
        return
    position = int(position)    

    # ABSOLUTE position displacement
    if axis == 'x':
        await m2.cmd_positionMirror.set_start(x=position)
    elif axis == 'y':
        await m2.cmd_positionMirror.set_start(y=position)
    elif axis == 'z':
        await m2.cmd_positionMirror.set_start(z=position)
   
    #awaiting until the position is stable
    interval = 5.0
    threshold = 2.0
    in_loop = True
    while in_loop:
       
        dof_meas = list()
        ti = time.time()
        while(time.time()-ti < interval):
            if axis == 'x':
                dof_meas.append(m2.tel_position.get().x)
            elif axis == 'y':
                dof_meas.append(m2.tel_position.get().y)
            elif axis == 'z':
                dof_meas.append(m2.tel_position.get().z)
               
        if abs(max(dof_meas)-min(dof_meas)) < threshold:
            in_loop = False
        await asyncio.sleep(5.0)


    # force threshold checks
    tangent_force_error = m2.tel_forceErrorTangent.get().force
    max_tangent = m2.tel_tangentForce.get().measured
    max_axial = m2.tel_axialForce.get().measured

    tangent_force_error = [abs(round(el, 2)) for el in tangent_force_error]
    max_tangent = [abs(round(el, 2)) for el in max_tangent]
    max_axial = [abs(round(el, 2)) for el in max_axial]

    sum_force = abs(m2.tel_forceErrorTangent.get().sum)
    weight_force = abs(m2.tel_forceErrorTangent.get().weight)

    # if the threshold is exceeded, then break
    if (max(tangent_force_error) > 950 or max(max_tangent) > 4870 or
    sum_force > 950 or weight_force > 1900 or max(max_axial) > 420):
        print('Some limit reached:')
        print(f'\nFmeasured: {str(max_tangent)} \nerror_tangent: {str(tangent_force_error)} \nsum: {str(sum_force)} \nweight: {str(weight_force)} \nmax_axial: {str(max(max_axial))}')
        return
    
    print(f'\nFmeasured: {str(max_tangent)} \nerror_tangent: {str(tangent_force_error)} \nsum: {str(sum_force)} \nweight: {str(weight_force)} \nmax_axial: {str(max(max_axial))}')
    
    await move_m2_rbp(axis)


In [ ]:
# X-DOF

axis='x'

await m2.cmd_positionMirror.set_start() #Restoring Zero position
await move_m2_rbp(axis)

In [ ]:
# Y-DOF

axis='y'

await m2.cmd_positionMirror.set_start() #Restoring Zero position
await move_m2_rbp(axis)

In [ ]:
# Z-DOF

axis='z'

await m2.cmd_positionMirror.set_start() #Restoring Zero position
await move_m2_rbp(axis)